Czyszczenie danych, preprocessing, inżynieria cech

In [21]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer

import torch
from torch.utils.data import Dataset

from transformers import AutoTokenizer

# wczytanie dancyh

In [22]:
train_df = pd.read_parquet("../data/processed/train.parquet")
val_df = pd.read_parquet("../data/processed/val.parquet")
test_df = pd.read_parquet("../data/processed/test.parquet")

train_df.shape, val_df.shape, test_df.shape

((1818, 10), (227, 10), (228, 10))

# Podział na X i y
Dzięki czemu mam spójne dane na wszystkich modelach

In [23]:
X_train = train_df["input_text"].astype(str)
X_val = val_df["input_text"].astype(str)
X_test = test_df["input_text"].astype(str)

y_train = train_df["label"].astype(int).to_numpy()
y_val = val_df["label"].astype(int).to_numpy()
y_test = test_df["label"].astype(int).to_numpy()

np.unique(y_train), np.unique(y_val), np.unique(y_test)

(array([0, 1, 2]), array([0, 1, 2]), array([0, 1, 2]))

# Mapowanie etykiet

In [24]:
id2label = {0: "low", 1: "mid", 2: "high"}
label2id = {v: k for k, v in id2label.items()}

id2label, label2id

({0: 'low', 1: 'mid', 2: 'high'}, {'low': 0, 'mid': 1, 'high': 2})

# TF-IDF

wybrałem TF-IDF, ponieważ daje silny baseline i działa bez GPU

In [25]:
tfidf = TfidfVectorizer(
    max_features=20000,
    ngram_range=(1, 2),
    min_df=2
)

X_train_tfidf = tfidf.fit_transform(X_train)
X_val_tfidf   = tfidf.transform(X_val)
X_test_tfidf  = tfidf.transform(X_test)

X_train_tfidf.shape, X_val_tfidf.shape, X_test_tfidf.shape


((1818, 5496), (227, 5496), (228, 5496))

# Dataset dla PyTorch
Klasa z konwersją per-batch w funkcji `__getitem__`

In [26]:
class TfidfDataset(Dataset):
    def __init__(self, X_sparse, y):
        self.X = X_sparse
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        x = self.X[idx].toarray().ravel().astype(np.float32)
        return torch.from_numpy(x), torch.tensor(self.y[idx], dtype==torch.long)


In [27]:
train_ds = TfidfDataset(X_train_tfidf, y_train)
val_ds = TfidfDataset(X_val_tfidf, y_val)
test_ds = TfidfDataset(X_test_tfidf, y_test)

len(train_ds), len(val_ds), len(test_ds)

(1818, 227, 228)

# Tokenizacja 

początkowo ustawiłem maks. długość na 256, ale po teście z komórki poniżej wynika, że można ustawić 128

In [28]:
# funkcja zwracająca stałe wartości dla tokenizowanych tekstów

def tokenize(texts):
    return tokenizer(
        list(texts),
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

# określenie modelu
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer("question [SEP] answer", truncation=True, max_length=128)


{'input_ids': [101, 3160, 102, 3437, 102], 'attention_mask': [1, 1, 1, 1, 1]}

# ustalenie `max_length` na bazie EDA

wynik array([32.  , 45.25, 57.  , 70.1 , 91.21]) mówi, że 99% danych mieści się w 91 tokenach. Czyli odpowiedzi są w większości dość zwięzłe.

Uzasadnia to wybranie długości maksymalnej tokenów w celu minimalizacji truncation, szybszego treningu i mniejszego zużycia VRAM

In [29]:
sample_texts = X_train.sample(200, random_state=67).tolist()
lengths = [len(tokenizer(t, truncation=False)["input_ids"]) for t in sample_texts]

np.percentile(lengths, [50, 75, 90, 95, 99])

array([32.  , 45.25, 57.  , 70.1 , 91.21])

## Podsumowanie etapu


- Przygotowano reprezentację TF-IDF dla modeli klasycznych i sieci neuronowej.
- Zaimplementowano kodowanie etykiet.
- Przeprowadzono tokenizację danych wejściowych dla modelu transformerowego.
- Dane są gotowe do etapu trenowania modeli.
